In [1]:
from autoritetsregister_oppslag import author_gender
import pandas as pd
import sqlite3 as sql


In [2]:
author_gender("Stensli , John Haakon")

[('Stensli, John Haakon', 'f', '1960-')]

Mye feil i MARC

In [3]:
df = pd.read_csv("/home/larsm/my_projects/metadata/Author_gender/author_gender_processed.csv")

In [4]:
df

,dhlabid,authors,gender_aut_reg,gender_genderguesser
0,100356077,"Olafsen , T.K.",Na,Na
1,100236738,"Buan , Edvard",Na,m
2,100221491,"Knausgård , Karl Ove",m,m
3,100402613,"Egner , Thorbjørn",m,m
4,100217239,"Nygårdshaug , Gert",m,m
...,...,...,...,...
505055,100354964,"Gjessing , Egil T.",Na,m
505056,100061598,"Knutson , Tormod",Na,m
505057,100480684,"Bygd , Sjur",m,m
505058,100087594,"Stensli , John Haakon / Fossum , Knut",f/Na,m/m


In [5]:
conn = sql.connect("/home/larsm/my_projects/metadata/Author_gender/author_gender.db")



In [7]:
df2 = pd.read_sql("select * from author_gender", conn)

In [11]:
df2.drop_duplicates(subset='dhlabid')

,dhlabid,oaiid,authors,gender
0,100356077,oai:nb.bibsys.no:999615795014702202,"Olafsen , T.K.",Na
1,100236738,oai:nb.bibsys.no:999429438054702202,"Buan , Edvard",Na
2,100082931,oai:nb.bibsys.no:999400358044702202,,Na
3,100221491,oai:nb.bibsys.no:990513939484702202,"Knausgård , Karl Ove",m
4,100402613,oai:nb.bibsys.no:999104475864702202,"Egner , Thorbjørn",m
...,...,...,...,...
570806,100354964,oai:nb.bibsys.no:999702982264702202,"Gjessing , Egil T.",Na
570807,100061598,oai:nb.bibsys.no:999706643204702202,"Knutson , Tormod",Na
570808,100480684,oai:nb.bibsys.no:998121178754702202,"Bygd , Sjur",m
570809,100087594,oai:nb.bibsys.no:999523174274702202,"Stensli , John Haakon / Fossum , Knut",f/Na


In [31]:
def parse_gender(df, idcol = 'dhlabid', author_col = 'authors', gender_col = 'gender'):
    """_summary_

    Args:
        df (pandas.DataFrame): _description_
        idcol (str, optional): _description_. Defaults to 'dhlabid'.
        author_col (str, optional): _description_. Defaults to 'authors'.
        gender_col (str, optional): _description_. Defaults to 'gender'.

    Returns:
        pandas.DataFrame: _description_
    """
    if df.index.name != idcol:
        df = df.set_index(idcol)
    
        df.drop_duplicates(subset='dhlabid')
    
        if len(df) > len(df.drop_duplicates(subset='dhlabid')):
            print("Dataframe contains duplicates")  
    
    aut = df[author_col].str.split("/", expand=True).stack()
    gen = df[gender_col].str.split("/", expand=True).stack()

    joined = aut.to_frame("authors").join(gen.to_frame("gender"))
    n = joined.authors.str.split(', ', expand=True)
    return joined

In [29]:
df2.index.name

'dhlabid'

In [38]:
gg = parse_gender(df, gender_col='gender_genderguesser')

In [33]:
auth_gend = parse_gender(df2)

In [36]:
auth_gend

authors gender
dhlabid                                     
100000000 0           Kinck , Hans E.      m
100000001 0             Holmsen , A.       f
          1               Strøm , Ole     Na
100000002 0  Vinsnes , Johan Frederik      m
100000005 0        Petersen , Fredrik     Na
...                               ...    ...
100579736 1            Dybwad , Jacob     Na
100579738 0                 Tank , C.      m
100579739 0            Rieck , Gustav     Na
100579740 0             Østbye , Tora     Na
100579741 0          Amundsen , Roald      m

[909172 rows x 2 columns]

In [37]:
auth_gend.gender.unique()

array(['m', 'f', 'Na', 'male'], dtype=object)

In [40]:
gg.sort_index()

authors gender
dhlabid                                     
100000000 0           Kinck , Hans E.      m
100000001 0             Holmsen , A.      Na
          1               Strøm , Ole      m
100000002 0  Vinsnes , Johan Frederik      m
100000005 0        Petersen , Fredrik      m
...                               ...    ...
100579736 1            Dybwad , Jacob      m
100579738 0                 Tank , C.     Na
100579739 0            Rieck , Gustav      m
100579740 0             Østbye , Tora      f
100579741 0          Amundsen , Roald      m

[842644 rows x 2 columns]

In [42]:
joined = auth_gend.join(gg, how='outer', lsuffix='_auth', rsuffix='_gg')

In [43]:
joined

authors_auth gender_auth                authors_gg  \
dhlabid                                                                       
100000000 0           Kinck , Hans E.           m           Kinck , Hans E.   
100000001 0             Holmsen , A.            f             Holmsen , A.    
          1               Strøm , Ole          Na               Strøm , Ole   
100000002 0  Vinsnes , Johan Frederik           m  Vinsnes , Johan Frederik   
100000005 0        Petersen , Fredrik          Na        Petersen , Fredrik   
...                               ...         ...                       ...   
100579736 1            Dybwad , Jacob          Na            Dybwad , Jacob   
100579738 0                 Tank , C.           m                 Tank , C.   
100579739 0            Rieck , Gustav          Na            Rieck , Gustav   
100579740 0             Østbye , Tora          Na             Østbye , Tora   
100579741 0          Amundsen , Roald           m          Amundsen , Roald   

            gender_gg  
dhlabid                
100000000 0         m  
100000001 0        Na  
          1         m  
100000002 0         m  
100000005 0         m  
...               ...  
100579736 1         m  
100579738 0        Na  
100579739 0         m  
100579740 0         f  
100579741 0         m  

[909172 rows x 4 columns]

In [55]:
disagreed = joined.loc[(joined.gender_auth != joined.gender_gg) & (joined.gender_auth == 'f')]

In [52]:
nonascii = joined.loc[joined.authors_auth.str.contains('ø|æ|å|Å|Ø|Æ')]

In [45]:
joined.loc[joined.gender_auth == joined.gender_gg]

,,authors_auth,gender_auth,authors_gg,gender_gg
dhlabid,,,,,
100000000,0,"Kinck , Hans E.",m,"Kinck , Hans E.",m
100000002,0,"Vinsnes , Johan Frederik",m,"Vinsnes , Johan Frederik",m
100000010,0,"Gundersen , B. (Bertrand)",Na,"Gundersen , B. (Bertrand)",Na
100000013,0,"Krogh , G.",Na,"Krogh , G.",Na
100000015,0,"Bang , Anthon",m,"Bang , Anthon",m
...,...,...,...,...,...
100579722,1,"Arbo , Peter Nicolai",m,"Arbo , Peter Nicolai",m
100579724,0,"Sørby , T. Ola",Na,"Sørby , T. Ola",Na
100579726,0,"Spurgeon , C.H.",Na,"Spurgeon , C.H.",Na


In [9]:
len(df2)

570811

In [10]:
len(df)

505060

In [53]:
author_gender("Bjørnson , Bjørnstjerne")

[('Sautreau, Dagny Bjørnson', 'f', '1876-1974')]

In [57]:
author_gender("Bjornson , Bjornstjerne")

[('Bjørnson, Bjørnstjerne', 'm', '1832-1910')]

In [54]:
author_gender("Jørgensen , S.E.")

[('Jørgensen, Dolly', 'f', '1972-'), ('Jørgensen, Jesper', '', '')]

In [62]:
author_gender("Jorgensen , S.E.")

[('Jørgensen, Dolly', 'f', '1972-')]

In [61]:
author_gender(" Bull , D.")

[('Bull-Gundersen, Anne B.', 'f', '1957-')]

In [59]:
author_gender("A. Thorkildsen")

[('Thorkildsen, Odd Inge Hesjedal', '', '')]